In [ ]:
#Load libs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from tysserand import tysserand as ty
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from PIL import Image, ImageOps
import fcsparser

from os import listdir
from os.path import isfile, join
#set up working dir

import sys
sys.path.extend([
    '../tysserand/tysserand',
    '../mosna',
])

import seaborn as sns
from time import time
import copy
from skimage import color
import matplotlib as mpl
import napari

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from scipy.stats import loguniform

import umap
# if not installed run: conda install -c conda-forge umap-learn
import hdbscan
from sklearn.cluster import OPTICS, cluster_optics_dbscan

from skimage import io

from scipy.stats import ttest_ind    # Welch's t-test
from scipy.stats import mannwhitneyu # Mann-Whitney rank test
from scipy.stats import ks_2samp     # Kolmogorov-Smirnov statistic

sys.path.append("/home/mouneem/mosna/")


from tysserand import tysserand as ty
from mosna import mosna
import glob
import re

In [ ]:
img_path = '/home/mouneem/tysserand/CORDELIER_PIERRE/C2v1/'

edg_path = '/home/mouneem/tysserand/CORDELIER_PIERRE/edg/'
coords_path = '/home/mouneem/tysserand/CORDELIER_PIERRE/CRDS/'
nets_path = '/home/mouneem/tysserand/CORDELIER_PIERRE/nets/'


imgs = [f for f in listdir(img_path) if isfile(join(img_path, f))]
coords = [f for f in listdir(coords_path) if isfile(join(coords_path, f))]
edges = [f for f in listdir(edg_path) if isfile(join(edg_path, f))]



In [ ]:
print(len(edges))

In [ ]:
for fileedg in edges[5:7]:
    print(fileedg)
    filecoords = "coords."+".".join(fileedg.split(".")[1:])
    filenet = ""+".".join(fileedg.split(".")[1:])
    pattern = filenet.split("czi")[0]
    edg = pd.read_csv(edg_path + fileedg , header = None)
    crd = pd.read_csv(coords_path + filecoords , header = None)
    plt.figure(figsize=(30, 30), dpi=80)
    crd=crd.drop([0,1])
    edg=edg.drop([0,1])
    img_found = pattern+"jpg" in imgs
    if img_found:
        img = plt.imread ( img_path + pattern+"jpg")
        fig, ax = ty.showim(img, figsize=(30, 30))
    else:
        fig, ax = plt.subplots(1,1,figsize=(30, 30))
    ax.scatter(crd.iloc[:,1], crd.iloc[:,2], c ="blue")
    Xs = list(edg.iloc[:,1 ])
    Ys = list(edg.iloc[:,1 ])
    print(Xs)

    print(type(edg))
    [x0, y0], [x1, y1] = coords[edg]
    ax.plot([x0, x1], [y0, y1], c=cmap(dist), zorder=0, alpha=alpha_edges, linewidth=linewidth)

    #fig.show()
    fig.savefig(nets_path+filenet+'.png')

    
    

        



In [ ]:
mosna_path = '/home/mouneem/tysserand/CORDELIER_PIERRE/Mixmat/'
mosna_output = str("/home/mouneem/tysserand/CORDELIER_PIERRE/mosna_output/")

mosnas = [f for f in listdir(mosna_path) if isfile(join(mosna_path, f))]
for mosnafile in mosnas[:2]:

    mixmat = pd.read_csv(mosna_path + mosnafile ,index_col=0 )
    print(mixmat)
    title = "Assortativity by cell types:"
    print(title)
    fig, ax = plt.subplots(figsize=(9, 6))
    sns.heatmap(mixmat, center=0, cmap="vlag", annot=True, linewidths=.5, ax=ax)
    plt.xticks(rotation=30, ha='right');
    # plt.xticks(rotation=30, ha='right', fontsize=20);
    # plt.yticks(fontsize=20);
    plt.savefig(mosna_output + mosnafile+"assortativity.png", bbox_inches='tight', facecolor='white')


In [ ]:
mosna_path = '/home/mouneem/tysserand/CORDELIER_PIERRE/Mixmat/'


In [ ]:
Layer1 = '/home/mouneem/tysserand/CORDELIER_PIERRE/tummors/'
tummors = [ ".".join(f.split(".")[:-3])  for f in listdir(Layer1) if isfile(join(Layer1, f))]
mosnas = [f for f in listdir(mosna_path) if isfile(join(mosna_path, f))]
mosna_output = str("/home/mouneem/tysserand/CORDELIER_PIERRE/mosna_output/")

print(file)

FullMatrix = MAT

for mosnafile in mosnas:
    file = ".".join(mosnafile.split(".")[1:-4])

    if file in tummors:
        print(file, mosnafile)

        mixmat = pd.read_csv(mosna_path + mosnafile ,index_col=0 )
        print(mixmat)

        keep = np.triu(np.ones(mixmat.shape)).astype('bool').reshape(mixmat.size)
        MAT = pd.DataFrame(mixmat.stack())
        MAT.to_csv('out.csv')

        MAT = pd.read_csv('out.csv')
        MAT.columns = ['X','Y','Value']

        di = {'C1': "Cancer", "C2": 'CD8 T-Cell', 'C3' : 'CD4 T-Cell', 'C4':'B Cell', 'Other':"Other",'C5':'CD3+CD20+' }
        MAT = MAT.replace( {"Y": di })
        MAT = MAT.replace( {"X": di })

        MAT["comb"] = MAT["X"].astype(str) + " / " + MAT["Y"].astype(str)
        
        MAT["Value"]=(MAT["Value"]-MAT["Value"].min())/(MAT["Value"].max()-MAT["Value"].min())

        MAT['sample'] = file
        FullMatrix = FullMatrix.append(pd.DataFrame(data = MAT))


print(FullMatrix)




In [ ]:
FullMatrix.to_csv('FullMatrix.csv')

FullMatrix = FullMatrix[['Value', 'comb', 'sample']]

FullMatrix2 = FullMatrix[ FullMatrix['comb'].isin(['Other / Other' , 'Other / CD8 T-Cell' , 'Other / CD4 T-Cell', 'Other / Cancer',
                                                    'CD8 T-Cell / CD8 T-Cell' , 'CD8 T-Cell / CD4 T-Cell', 'CD8 T-Cell / Cancer',
                                                    'CD4 T-Cell / CD4 T-Cell', 'CD4 T-Cell / Cancer',
                                                    'Cancer / Cancer', ]) ]

FullMatrix2.index = FullMatrix2[['sample']]
print(FullMatrix2)

Matrix = FullMatrix2.pivot_table(index=["sample"], 
                    columns='comb', 
                    values='Value')


plt.figure(figsize=(20, 5))

sns.heatmap(Matrix)

In [ ]:
plt.figure(figsize=(20, 5))
sns.clustermap(Matrix, yticklabels=False, center = 0, z_score =1)